In [1]:
import xarray
import numpy as np
import zarr
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster, progress
from rechunker import rechunk
from tqdm import tqdm
import gc
import os

In [2]:
from dask.distributed import Client
client = Client(n_workers=3)

In [3]:
import shutil
import gc

def rechunk_zarr_store(zarr, chunks, target_zarr, steps = 200, iterate_over='time'):
    slice_lengths = int(len(zarr[iterate_over]) / steps)
    if os.path.exists(target_zarr):
        temp = xarray.open_zarr(target_zarr)
        already_rechunked = len(temp[iterate_over])
        temp.close()
    else:
        already_rechunked = 0
    
    # already_rechunked =0
    for i in tqdm(range(0, len(zarr[iterate_over]), slice_lengths)):
        if i + slice_lengths <= already_rechunked:
            print(f'skipped {i}')
            continue

        if i < already_rechunked and i + slice_lengths > already_rechunked:
            zarr_slice = zarr.isel({iterate_over : slice(already_rechunked, i+slice_lengths)})
        else:
            zarr_slice = zarr.isel({iterate_over :slice(i, i + slice_lengths)})
        
        for v in list(zarr_slice.coords.keys()):
            if zarr_slice.coords[v].dtype == object:
                zarr_slice[v].encoding.clear()
                
        for var in zarr_slice:
            zarr_slice[var].encoding.clear()
        

        zarr_slice = zarr_slice.chunk(chunks)
            
        if i == 0:
            zarr_slice.to_zarr(target_zarr, mode="w")
        else:
            zarr_slice.to_zarr(target_zarr, append_dim=iterate_over)
        del zarr_slice
        gc.collect()

In [5]:
sarah = xarray.open_zarr('/scratch/snx3000/kschuurm/ZARR/SARAH3_new.zarr')
sarah

<xarray.Dataset>
Dimensions:       (channel: 4, time: 139915, lat: 658, lon: 736)
Coordinates:
  * channel       (channel) object 'CAL' 'KI' 'SID' 'SIS'
  * lat           (lat) float32 28.98 29.02 29.08 29.12 ... 61.72 61.78 61.83
  * lon           (lon) float32 -7.775 -7.725 -7.675 ... 28.88 28.92 28.98
  * time          (time) datetime64[ns] 2015-01-01 ... 2023-01-01T23:30:00
Data variables:
    channel_data  (channel, time, lat, lon) float16 dask.array<chunksize=(4, 1, 658, 736), meta=np.ndarray>
    max_x         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    max_y         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    min_x         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    min_y         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes: (12/41)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    CDO:                        Climate Data Operators version 1.9.10 (https:...
    Conventions:                CF-1.7,ACDD-1.3
    creator_email:              contact.cmsaf@dwd.de
    creator_name:               DE/DWD
    creator_type:               institution
    ...                         ...
    time_coverage_duration:     P1D
    time_coverage_end:          2023-01-01T00:00:00
    time_coverage_resolution:   PT30M
    time_coverage_start:        2022-12-31T00:00:00
    title:                      CM SAF Surface Solar Radiation Climate Data R...
    variable_id:                CAL

In [6]:
target_zarr = '/scratch/snx3000/kschuurm/ZARR/SARAH3_timeseries_chunk.zarr'

chunks = {'channel':-1, 'time':-1, 'lat':100, 'lon':100}

rechunk_zarr_store(sarah, chunks, target_zarr, steps=50)

100%|██████████| 51/51 [27:00<00:00, 31.78s/it]


In [7]:
sarah = xarray.open_zarr( '/scratch/snx3000/kschuurm/ZARR/SARAH3_timeseries_chunk.zarr')
sarah

<xarray.Dataset>
Dimensions:       (channel: 4, time: 139915, lat: 658, lon: 736)
Coordinates:
  * channel       (channel) object 'CAL' 'KI' 'SID' 'SIS'
  * lat           (lat) float32 28.98 29.02 29.08 29.12 ... 61.72 61.78 61.83
  * lon           (lon) float32 -7.775 -7.725 -7.675 ... 28.88 28.92 28.98
  * time          (time) datetime64[ns] 2015-01-01 ... 2023-01-01T23:30:00
Data variables:
    channel_data  (channel, time, lat, lon) float16 dask.array<chunksize=(4, 2798, 100, 100), meta=np.ndarray>
    max_x         (time) int64 dask.array<chunksize=(2798,), meta=np.ndarray>
    max_y         (time) int64 dask.array<chunksize=(2798,), meta=np.ndarray>
    min_x         (time) int64 dask.array<chunksize=(2798,), meta=np.ndarray>
    min_y         (time) int64 dask.array<chunksize=(2798,), meta=np.ndarray>
Attributes: (12/41)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    CDO:                        Climate Data Operators version 1.9.10 (https:...
    Conventions:                CF-1.7,ACDD-1.3
    creator_email:              contact.cmsaf@dwd.de
    creator_name:               DE/DWD
    creator_type:               institution
    ...                         ...
    time_coverage_duration:     P1D
    time_coverage_end:          2023-01-01T00:00:00
    time_coverage_resolution:   PT30M
    time_coverage_start:        2022-12-31T00:00:00
    title:                      CM SAF Surface Solar Radiation Climate Data R...
    variable_id:                CAL

In [4]:

sarah = xarray.open_zarr( '/scratch/snx3000/kschuurm/ZARR/SARAH3_timeseries_chunk.zarr')
target_zarr =  '/scratch/snx3000/kschuurm/ZARR/SARAH3_timeseries_chunk2.zarr'

chunks = {'channel':-1, 'time':-1, 'lat':25, 'lon':25}

rechunk_zarr_store(sarah, chunks, target_zarr, steps=50, iterate_over='lat')

100%|██████████| 51/51 [19:38<00:00, 23.10s/it]
